In [1]:
%%javascript
MathJax.Hub.Config({TeX:{equationNumbers:{autoNumber:"AMS"}}});

<IPython.core.display.Javascript object>

# BCB Final Project - a.y. 17/18

### <i>Fiaschi Lorenzo, Franco Danilo</i>

<ol> <b>TODO</b>
    <li><strike>problem presentation (dataset, objectives)</strike></li>
    <li><strike>preprocessing</strike></li>
    <li><strike>mkl procedure overview</strike></li>
    <li><strike>chosen model description (alignment model, cortes approach)</strike></li>
    <li><strike>algorithm pipeline (with a major point in the cross-validation decisions, how it is shuffled...)</strike></li>
    <li><strike>approach settings</strike></li>
    <li>performances over toy (with description on how it is generated</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>performances over real dataset</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>what we can learn from the best configuration (sparsity)</li>
    <li>kernel integration on the only meaningful dataset (clinical)</li>
    <li>overview on the regression approach</li>
    <li>regression performances with data integration</li>
    <li>single kernel approach after regression results</li>
    <li>final comments over the results</li>
</ol>

### 1. The Problem
The dataset consists of 2741 aligned patients affected by Diabetes; it is structured in 3 differents tables:
<ol>
    <li><b>Genetic Features</b>, 347 - Related to the SNP of 344 genes + 3 composite genes scores [blood pressure, alzheimer, CVD]</li>
    <li><b>Retina's Features</b>, 157 - Engineered features extracted from patients' retinas</li>
    <li><b>Clinical Features</b>, 15 - General patients' information</li>
</ol>
The objective of the study is to find a model that is able to predict whether a patient will incurr in either cardiovascular failures or in episodes of dementia.
In the positive case, it is also aked to predict at which age this episodes will occurr.
In order to learn such model, the output dataset has been provided; it is composed by two information, both for the occurrence of a cardiovascular failure and for the dementia episode: whether the episode has happened or not and at what age.

### 2. Data Preprocessing
In order to be fed, the tables have been cleaned as follows:
<ul>
    <li>
        <b><i>NaN Filling</i></b>: 
        <ul>
            <li>
                Clinical DS: 
                <ul>
                    <li>
                        ["therapy","gender","precedent CVD", "smoker"] $\rightarrow$ most frequent;
                    </li>
                    <li>
                        Others $\rightarrow$ mean.
                    </li>
                </ul>
            <li>
                Genetic DS: 
                <ul>
                    <li>
                        [composite gene scores] $\rightarrow$ mean;
                    </li>
                    <li>
                        Others $\rightarrow$ min.
                    </li>
                </ul>
            <li>
                Vampire DS: 
                <ul>
                    <li>
                        All $\rightarrow$ mean.
                    </li>
                </ul>
        </ul>
    </li>
    <li>
        <b><i>One-Hot Encoding</i></b>:
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        "therapy" and "Apoe4Presence".
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li><b><i>Boolean: from binary to symmetric</i></b>: 
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        ["gender","precedent CVD", "smoker"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
            <li>Outputs:
                <ul>
                    <li>
                        ["cvd_fail","dement_fail"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ul>

### 3. Multiple Kernel Learning
<b>Multiple Kernel Learning</b> refers to a set of machine learning methods that use a predefined set of kernels and learn an optimal linear or non-linear combination of kernels as part of the algorithm.<br>
Reasons to use multiple kernel learning include:
<ul>
    <li>the ability to select for an optimal kernel and parameters from a larger set of kernels, reducing bias due to kernel selection while allowing for more automated machine learning methods;
    </li>
    <li>combining data from different sources (e.g. sound and images from a video) that have different notions of similarity and thus require different kernels;
    </li>
</ul>
<img src='others/mkl.png'>

### 4. The Model
<ul>
    <li><b><i>Target Function</i></b> $\rightarrow$ Similarity-based function; it uses a similarity metric between the combined kernel matrix and an optimum kernel matrix calculated from the training data, in order to select the combination function parameters that maximize the similarity. The similarity between two kernel matrices can be calculated using kernel alignment, Euclidean distance, Kullback-Leibler (KL) divergence, or any other similarity measure.
    </li>
    <li><b><i>Training Method</i></b> $\rightarrow$ One-step method; it calculates both the combination function parameters and the parameters of the combined base learner in a single pass. One can use a sequential approach or a simultaneous approach. In the sequential approach, the combination function parameters are determined first, and then a kernel-based learner is trained using the combined kernel. In the simultaneous approach, both set of parameters are learned together.
    </li>
</ul>
The chosen algorithm is called <i>Centered-kernel alignment</i> [1] and its main purpose is to compute a set of $P$ kernels $K_i$, $i \, = \, 1...P$ from the datasets and to use a linear combination of them to approximate an ideal kernel. Then the obtained approximated kernel can be used for both classification and regression purposes. The functional is:<br>

\begin{equation}
\max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK)
\label{eq:problem}
\end{equation}

where $IK = y^T y$ is the ideal kernel, $K_{\eta} = \sum\limits_{i=1}^P K_i \eta_i$ is its approximation and $\mathcal{M}=\{\eta : ||\eta||_2 = 1\}$ imposing $\eta$ being a unit norm vector. In turn, $CA(K_1, K_2)$ is defined as

\begin{equation*}
CA(K_1^c, K_2^c) = \frac{\langle K_1^c, K_2^c \rangle_F}{\sqrt{\langle K_1^c, K_1^c\rangle_F \; \langle K_2^c, K_2^c \rangle_F}}
\end{equation*}

with $K^c$ is the centered version of $K$ and can be calculated as

\begin{equation*}
K^c = K - \frac{1}{N} 11^TK - \frac{1}{N} K11^T + \frac{1}{N^2} \left(1^TK1\right)11^T
\end{equation*}

where $1$ is the vector of ones with proper dimension.

The optimization problem \eqref{eq:problem} has then a unique analytical solution

\begin{equation}
\eta = \frac{M^{-1}a}{||M^{-1}a||_2}
\label{eq:problem_solution}
\end{equation}

where $M = \{\langle K_m, K_h\rangle_F\}_{m,h \, = \, 1}^P$ and $a = \{\langle K_m, IK\rangle_F\}_{m \, = \, 1}^P$.

In order to avoid overfitting, to be more robust in the learning procedure and in order to get deeper insights of the problem we added either an $L_1$ or $L_2$ penalty to the functional \eqref{eq:problem}, obtaining the following two reformulations

\begin{equation}
\max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK) - \lambda ||\eta||_2^2 \;\; (a), \qquad \max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK) - c ||\eta||_1 \;\; (b).
\label{eq:problem_penalty}
\end{equation}

While the Equation \eqref{eq:problem_penalty}(a) has the unique analytical solution 

\begin{equation}
\eta = \frac{(M-\lambda I)^{-1}a}{||(M-\lambda I)^{-1}a||_2}
\label{eq:problem_solution_lambda}
\end{equation}

Equation \eqref{eq:problem_penalty}(b) does not due to the non-differentiability of the $L_1$ norm. To converge to one of the optima we applied the forward-backward splitting method [2][3].

### 5. The Algorithm

<ol>
    <li><i>Datasets loading</i> $\rightarrow$ (Clinical, Genetic, Vampire, Outputs);</li>
    <li><i>Outer split</i> $\rightarrow$ 75% training + 25% test for final testing purposes;</li>
    <li><i>Kernel definition</i> $\rightarrow$ dictionary list: [{<b>kernelType1:[parameter list]</b>, <b>kernelType2:[parameter list]</b>, <b>...</b> }, <b>...</b> ] </li>
    <li><i>Sampling rounds</i> $\rightarrow$ 3 rounds of 75% training + 25% mid test (extracted from the previous 75% of training) for statistical stability:
        <ol>
            <li>If stated, perform training matrix centering and/or normalisation;</li>
            <li>For each dictionary configuration:
                <ol>
                    <li><i>If regression</i>: Normalize the sampling training outputs (store aside one non normalised version);</li>
                    <li>Build the ideal kernel from the training outputs;</li>
                    <li>Build all the possible combination between the kernels parameters;</li>
                    <li>3-Fold cross validation over the 75% sampling training:
                        <ol>
                            <li><i>If regression</i>: Normalize the sampling training and validation outputs with their respective 2-norms;</li>
                            <li>Build the ideal kernel matrices from both training and validation output sets;</li>
                            <li>For each configurations:
                                <ol>
                                    <li>Compute all the kernel matrices for that configuration;</li>
                                    <li>Find the optimal weights for those kernel matrices using the ideal training kernel;</li>
                                    <li>Compute the weighted sum of the kernel matrices;</li>
                                    <li>Compare the obtained matrix with the ideal validation kernel using the Cortes Alignment metric and store this similarity measure;</li>
                                </ol>
                            </li>
                        </ol>                       
                    </li>
                    <li>Find the configuration with greatest mean across the three validation fold;</li>
                    <li>Recompute weights and similarity score for the newfound setting using the ideal training kernel found at poin <i>b</i>;</li>
                    <li>Compute accuracy, precision and recall (<i>classification</i>) or average error and variance (<i>regression</i>) for the selected setting against the sampling test set;</li>
                    <li>Store these value, decorated with configuration and alignment for this dictionary;</li>
                </ol>
            </li>
        </ol>
    </li>
    <li>For each sample round, sum all the alignment values related to the same kernel parameters in order to find the configuration that best behaved, keeping the dictionary within a list separated;</li>
    <li>Try the configurations (one per dictionary) against the outer test set, compute accuracy, precision and recall (<i>classification</i>) or average error and variance (<i>regression</i>)</li>
</ol>

### 6. Approach Setting

Kernel used:
    
<ul>
    <li>Gaussian: $\hspace{4mm}K_{\sigma}(x_i, x) = e^{\,-\,\left(\frac{||x_i \, - \, x||^2}{2\sigma}\right)}$</li>
    <li>Linear: $\hspace{9mm}K(x_i, x) = x_i \cdot x^T$</li>
    <li>Polynomial: $\hspace{1mm}K_d(x_i, x) = (1 +  x_i \cdot x^t)^d$</li>
    <li>Laplacian: $\hspace{3mm}K_{\gamma}(x_i, x) = e^{\,-\,(\, \gamma \, ||x_i \, - \, x||_1)}$</li>
    <li>Sigmoid: $\hspace{5mm}K_{\gamma}(x_i, x) = \tanh \,(\,\gamma \, \langle X, Y \rangle + 1)$</li>
</ul>

The basical approach exploited has been picking three different kernels and applying them to every dataset, getting back 9 kernel matrices. Several combinations of three data preprocessing methods (Data origin centering, Data Normalization, Kernel Normalization) have been tested.

### 7. Toy Testing

In order to analyze the correctness of the algorithm implemented three synthetic datasets have been generated and the algorithm have been launched over them.


#### 7.1 Toy Dataset Generation

For dataset generation we used the sklearn.datasets.make_classification function [4], initialized with the following configuration:

<ul>
    <li>n_samples = 300</li>
    <li>n_features = 30</li>
    <li>n_informative = 10</li>
    <li>n_redundant = 0</li>
    <li>n_classes = 2</li>
</ul>

The data gathered in this way have been splitted in three datasets and the informative variables have been distributed evenly between the first two. This procedure let us to check if the algorithm is able or not to percieve the irrelevance of the third dataset.

Then, the three datasets have been splitted in training and test sets exploiting sklearn.model_selection.StratifiedShuffleSplit [5]. The initialization in the next

<ul>
    <li>n_splits = 1</li>
    <li>test_size = 0.25</li>
</ul>


#### 7.2 Performances over Toy

In the next the results achieved using the Toy datasets are shown. In particular, they are compared with the learning performances got studying one single dataset (the first one) using common learning algorithm, e.g., Logistic Regression and SVM.

<table style="width:100%">
    <tr>
        <th> 
          <p><font  color="red">Based on CA</font></p>
        </th>
        <th align="justify">
          Cardio configuration
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Cardio train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;\;$ Normalization
        </th>
        <td align="left">
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
</table>

<br>
<br>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Dementia configuration
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Dementia train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Polynomial : 7, 2, 2<br>
            Gaussian: 0.3, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.8140113798008535<br>
            Precision: 0.7804878048780488<br>
            Recall: 0.8648648648648649
        </td>
        <td>
            CA: 0.21867020939197357<br>
            Accuracy: 0.8483169129720854<br>
            Precision: 0.8315456327135972<br>
            Recall: 0.8690476190476191
        </td>
        <td>
            [1.37635178e-06, 9.93310800e-02, 9.70133466e-03, 6.67864067e-01, 4.38510115e-02, -7.36256679e-01]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td>
            Lambda: 0.7<br>
            Polynomial : 2, 2, 7<br>
            Gaussian: 0.3, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.8275248933143671<br>
            Precision: 0.7857142857142857<br>
            Recall: 0.8918918918918919
        </td>
        <td>
            CA: 0.2320201774307744<br>
            Accuracy: 0.8249178981937603<br>
            Precision: 0.8088881826367921<br>
            Recall: 0.8452380952380952
        </td>
        <td>
            [0.69048272, -0.40438296, 0.33601634, 0.23292882, 0.41944434, 0.12888623]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td>
            Lambda: 1.3<br>
            Sigmoid: 0.3, 0.2, 0.1<br>
            Polynomial: 2, 3, 3<br>
            Gaussian: 0.2, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.7339971550497866<br>
            Precision: 0.7073170731707317<br>
            Recall: 0.7837837837837838
        </td>
        <td>
            CA: 0.23112759419514883<br>
            Accuracy: 0.8013136288998358<br>
            Precision: 0.7902298850574713<br>
            Recall: 0.8095238095238094
        </td>
        <td>
            [0.00387049, 0.0155308, 0.00115572, 0.00295326, 0.07049194, 0.00107543, 0.00713828, 0.99735214, 0.00110999]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td>
            Lambda: 0.5<br>
            Polynomial: 2, 7, 7<br>
            Gaussian: 0.6, 0.6, 0.6
        </td>
        <td>
            Accuracy: 0.8666429587482219<br>
            Precision: 0.8684210526315785<br>
            Recall: 0.8648648648646
        </td>
        <td>
            CA: 0.21082589906567226<br>
            Accuracy: 0.8364121510673236<br>
            Precision: 0.8265773453945497<br>
            Recall: 0.8452380952380952
        </td>
        <td>
            [0.28293068, 0.44627082, 0.14576441, 0.44132354, 0.55313343, 0.44589491]
        </td>
    </tr>
</table>


#### 7.3 Results discussion

### 8 Real Data

In the next, the results achieved over the real data are shown and discussed. Again the comparison between the data integration approach and the shallow learning is proposed.


#### 8.1 Performances


<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on CA</font ></p>
        </th>
        <th align="justify">
          Cardio configuration
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Cardio train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Laplacian: 0.4, 0.9, 0.9<br>
            Gaussian: 0.5, 1, 1
        </td>
        <td>
            Accuracy: 0.4876963740600104<br>
            Precision: 0.34516765285996054<br>
            Recall: 0.7231404958677686
        </td>
        <td>
            CA: 0.03207411722599631<br>
            Accuracy: 0.5575231962332087<br>
            Precision: 0.370735210200490<br>
            Recall: 0.606425702811245
        </td>
        <td>
            [-0.80895634, 0.2557566, 0.26466149, 0.26465771, 0.26465911, 0.26465895]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td align="left">
            Lambda: 0.9<br>
            Sigmoid: 0.6, 0.6, 0.2<br>
            Gaussian: 0.6, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.5283299828754374<br>
            Precision: 0.3825503355704698<br>
            Recall: 0.47107438016528924
        </td>
        <td>
            CA: 0.023241379530336285<br>
            Accuracy: 0.48841919401744915<br>
            Precision: 0.3147208168722621<br>
            Recall: 0.6004016064257028
        </td>
        <td>
            [-4.04835811e-11, 6.39532405e-01, -4.04835893e-11, 4.69236031e-01, -4.04835804e-11, 6.08946509e-01]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 0.5<br>
            Sigmoid: 0.9, 0.9, 0.9<br>
            Gaussian: 0.5, 1, 1
        </td>
        <td>
            Accuracy: 0.49077693395875216<br>
            Precision: 0.34829721362229105<br>
            Recall: 0.9297520661157025
        </td>
        <td>
            CA: 0.024054921569728476<br>
            Accuracy: 0.58211581960024<br>
            Precision: 0.39161162087370743<br>
            Recall: 0.6345381526104418
        </td>
        <td>
            [0, -0.57734802, 0, -0.5773513, 0, -0.57735149]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td align="left">
            Lambda: 0.9<br>
            Polynomial: 2, 2, 2<br>
            Gaussian: 0.6, 0.6, 0.3
        </td>
        <td>
            Accuracy: 0.48501600774328046<br>
            Precision: 0.3414043583535109<br>
            Recall: 0.5826446280991735
        </td>
        <td>
            CA: 0.022394111162592222<br>
            Accuracy: 0.500525088861192<br>
            Precision: 0.3251298444956087<br>
            Recall: 0.5020080321285141
        </td>
        <td>
            [-0.00216869, -0.57733688, -0.00596827, -0.57733877, -0.00161064, -0.57733799]
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on CA</font></p>
        </th>
        <th>
          Dementia configuration
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Dementia train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Sigmoid: 0.2, 0.6, 0.6<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.5145762008507107<br>
            Precision: 0.1783876500857633<br>
            Recall: 0.8739495798319328
        </td>
        <td>
            CA: 0.025238034586317307<br>
            Accuracy: 0.5041506939854594<br>
            Precision: 0.1744290781439166<br>
            Recall: 0.7902621722846442
        </td>
        <td>
            [0.00000000e+00, -4.22882481e-01, -1.08259827e-16,  6.46392606e-01, 0.00000000e+00, 6.35096061e-01]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td align="left">
            Lambda: 0.5<br>
            Sigmoid: 0.9, 0.4, 0.4<br>
            Gaussian: 1, 0.5, 1
        </td>
        <td>
            Accuracy: 0.5296711277103434<br>
            Precision: 0.1918238993710692<br>
            Recall: 0.5126050420168067
        </td>
        <td>
            CA: 0.026724874653312764<br>
            Accuracy: 0.4997708746419916<br>
            Precision: 0.1696459702387958<br>
            Recall: 0.5430711610486891
        </td>
        <td>
            [-3.76186696e-10, 5.70377232e-01, -3.76628319e-10, 5.80245771e-01, -3.76186694e-10, 5.81364480e-01]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 0.7<br>
            Laplacian: 0.9, 0.4, 0.4<br>
            Gaussian: 0.5, 1, 0.5
        </td>
        <td>
            Accuracy: 0.5208527855586679<br>
            Precision: 0.18006430868167203<br>
            Recall: 0.9411764705882353
        </td>
        <td>
            CA: 0.025731949505026247<br>
            Accuracy: 0.5937959903062349<br>
            Precision: 0.21972420698129522<br>
            Recall: 0.7303370786516853
        </td>
        <td>
            [-0.40824829, -0.40825135, -0.40824791, -0.40824741, -0.40824731, -0.40824747]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td align="left">
            Lambda: 0.5<br>
            Polynomial: 2, 7, 7<br>
            Gaussian: 0.6, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.48646125116713346<br>
            Precision: 0.16621253405994552<br>
            Recall: 0.5126050420168067
        </td>
        <td>
            CA: 0.02512163251299737<br>
            Accuracy: 0.5154571491517955<br>
            Precision: 0.18188049096731554<br>
            Recall: 0.543071161048689
        </td>
        <td>
            [-0.00255566, -0.57702685, -0.03326846, -0.57702322, -0.00376406, -0.57702403]
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Cardio configuration
        </th>
        <th>
          Cardio test
        </th>
        <th>
          Cardio train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Linear: -<br>
            Gaussian: 0.3, 0.6, 0.6
        </td>
        <td>
            Accuracy: 0.53740443386484<br>
            Precision: 0.69340974212034384<br>
            Recall: 0.54077253212034384
        </td>
        <td>
            CA: 0.0.025411941604895932<br>
            Accuracy: 0.5351351041379944<br>
            Precision: 0.38918226446830567<br>
            Recall: 0.33134920634920634
        </td>
        <td>
            [-1.82364335e-01, -2.09411395e-04, -9.83230958e-01, -2.09400783e-04, -5.91624351e-08, -2.09398176e-04]*
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td>
            Lambda: 0.7<br>
            Polynomial: 5, 3, 3<br>
            Gaussian: 0.5, 0.5, 1
        </td>
        <td>
            Accuracy: 0.6064102928497666<br>
            Precision: 0.43137254901960786<br>
            Recall: 0.6609442060085837
        </td>
        <td>
            CA: 0.022862124927406354<br>
            Accuracy: 0.5194857326360217<br>
            Precision: 0.3415364884671758<br>
            Recall: 0.6150793650793651
        </td>
        <td>
            [9.99997211e-01, 7.87108913e-04, -7.21686219e-04, 1.48091632e-03, -9.39626147e-05, 1.49520357e-03]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td>
            Lambda: 0.5<br>
            Sigmoid: 0.6, 0.2, 0.2<br>
            Gaussian: 0.3, 0.3, 0.6
        </td>
        <td>
            Accuracy: 0.5272108843537415<br>
            Precision: 0.3716216216216216<br>
            Recall: 0.8979591836734694
        </td>
        <td>
            CA: 0.02453613516429783<br>
            Accuracy: 0.621903852276345<br>
            Precision: 0.42268894019014386<br>
            Recall: 0.6888888888888888
        </td>
        <td>
            [ 0, -0.57741441, 0, -0.57732454, 0, -0.57731185]
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td>
            Lambda: 0.7<br>
            Sigmoid: 0.6, 0.2, 0.2<br>
            Gaussian: 0.3, 0.3, 0.3
        </td>
        <td>
            Accuracy: 0.52717687074829932<br>
            Precision: 0.6612903225806452<br>
            Recall: 0.4857142857142858
        </td>
        <td>
            CA: 0.028289233641315092<br>
            Accuracy: 0.5464762235535875<br>
            Precision: 0.36338722125747674<br>
            Recall: 0.5494949494949496
        </td>
        <td>
            [0,  0.57739099, 0, 0.57733643, 0, 0.57732339]*
        </td>
    </tr>
</table>

<table style="width:100%">
    <tr>
        <th>
          <p><font  color="red">Based on Accuracy</font></p>
        </th>
        <th align="justify">
          Dementia configuration
        </th>
        <th>
          Dementia test
        </th>
        <th>
          Dementia train
        </th>
        <th>
          Eta
        </th>
    </tr>
    <tr>
        <th>
            L2 - Centering - $\;\;$ Normalization
        </th>
        <td align="left">
            Lambda: 1.3<br>
            Laplacian: 0.4, 0.9, 0.4<br>
            Gaussian: 1, 1, 0.5
        </td>
        <td>
            Accuracy: 0.49921840629767955<br>
            Precision: 0.34791252485089463<br>
            Recall: 0.7322175732217573
        </td>
        <td>
            CA: 0.03306303924104954<br>
            Accuracy: 0.5441536259982459<br>
            Precision: 0.3671488308585083<br>
            Recall: 0.5040160642570282
        </td>
        <td>
            [-0.82520508, 0.25346384, 0.25237393, 0.25236869, 0.25242335, 0.25237385]
        </td>
    </tr>
    <tr>
        <th>
            L2 - Centering -$\;$K Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering - $\;\;$ Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
    <tr>
        <th>
            L1 - Centering -$\;$K Normalization
        </th>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
        <td>
            v
        </td>
    </tr>
</table>


#### 8.2 Results discussion

### 9 Further considerations and studies

From the Section 8 has been inferred the with high probability the most (and only!) meaningful dataset for the project purposes (see Section 1) is the clinical one. Because of this an interesting study could have been the one which applies a multi-kernel learning procedure only to such dataset. This kind of approach let the realization of multiple points of view (one per kernel) of the same object, i.e., the clinical dataset. Thus, the wider overall vision of the data got could provide a better comprehension of the correlation existing between the data and the diseases.


#### 9.1 MKL over the Clinical dataset

The study has been realized applying an MKL procedure to the Clinical dataset. The kernels used are all the ones listed in Section 6. In the next the performances.


#### 9.2 Results discussion

### 10 Shallow regression

Once enlightened as the most (and only) relevant dataset for dementia and cardiovascular failures, the Clinical one has been exploited for regression purposes. The regressive learning has the goal to predict the age at which the diesease will arise.
The regression algorithm adopted are Lasso and SVM. In the next the performances and their discussion.


#### 10.1 Shallow regression performances



#### 10.2 Results discussion

### 11 Conclusions and further possible studies

In this section is presented a summarization of the results that have been obtained during the study presented above. Further possible study are suggested. Most of the ideas have not been tested largely due to computational limits of the authors.


#### 11.1 What has been learned


#### 11.2 Best configurations


#### 11.3 Suggested further studies

### References
[1] Corinna Cortes, Mehryar Mohri, and Rostamizadeh Afshin. "Two-stage learning kernel algorithms", In Proceedings of the 27th International Conference on Machine Learning.<br>
[2] https://en.wikipedia.org/wiki/Proximal_gradient_methods_for_learning<br>
[3] Combettes, Patrick L.; Wajs, Valérie R. (2005). "Signal Recovering by Proximal Forward-Backward Splitting", Multiscale Model, Simul 4 (4): 1168–1200. doi:10.1137/050626090<br>
[4] http://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html<br>
[5] http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html